## Setup

In [87]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Dependencies

In [88]:
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [89]:
import datetime as dt
import os

import ipywidgets as widgets
from IPython.display import display,clear_output

In [114]:
import pandas as pd
pd.options.mode.chained_assignment = None   # suppresses SettingWithCopyWarning

In [90]:
pwd

'/home/jake/Projects/mila/6758/Docker/notebooks'

In [91]:
from ift6758.client import GameTracker, NHLAPIClient, PredictorAPIClient
from ift6758.client.nhl_api_client import UnknownGameException
from ift6758.client.predictor_api_client import UnknownModelException

### Setup

In [92]:
if not os.path.exists("./tmp_data/"):
    os.makedirs("./tmp_data/")

In [93]:
nhl_api = NHLAPIClient(raw_game_directory="./tmp_data/")
predictor = PredictorAPIClient()
game_tracker = GameTracker(nhl_api_client=nhl_api, predictor_client=predictor)

### Dashboard Actions

In [116]:
update_dashboard = True
game_id = 2021020008
model_id = "xgboost-feats-non-corr"   # nn-adv, xgboost-lasso logistic-regression-distance-and-angle
# not working models: xgboost-lasso, nn-adv
if update_dashboard:
    try:
        game_tracker.update(game_id, model_id)
    except UnknownGameException as uge:
        print(f"Game ID {game_id} not recognized, so dashboard unchanged")
    except UnknownModelException as ume:
        print(f"Dashboard unchanged because model_id " + str(ume))

    events = game_tracker.get_events_and_predictions()

    most_recent_event = events.head(1).squeeze()
    home_team = most_recent_event['home_team']
    away_team = most_recent_event['away_team']

    home_xG = round(events.loc[events['shooter_team_name']==home_team, 'predictions'].sum(), 2)
    away_xG = round(events.loc[events['shooter_team_name']==away_team, 'predictions'].sum(), 2)

    print(f"Period {most_recent_event['period']}, {most_recent_event['time_remaining']} Remaining")
    print(f"Home: {home_team} Away: {away_team}")
    print(f"Home G: {most_recent_event['goals_home']} Away G: {most_recent_event['goals_away']}")
    print(f"Home xG: {home_xG} Away xG: {away_xG}")
    print(f"Recent events:")
    display(events[['type','description', 'predictions']].head(15))


Period 3, 00:00 Remaining
Home: Ottawa Senators Away: Toronto Maple Leafs
Home G: 3 Away G: 2
Home xG: 4.31 Away xG: 4.77
Recent events:


/home/jake/Projects/mila/6758/Docker/ift6758/ift6758/client/GameTracker.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_pred_inputs = game_events.loc[new_event_idx][shot_mask]


,type,description,predictions
347,GAME_OFFICIAL,Game Official,NaN
346,GAME_END,Game End,NaN
345,PERIOD_OFFICIAL,Period Official,NaN
344,PERIOD_END,End of 3rd Period,NaN
343,MISSED_SHOT,Morgan Rielly Over Net Anton Forsberg,NaN
342,HIT,Nick Paul hit William Nylander,NaN
341,FACEOFF,David Kampf faceoff won against Shane Pinto,NaN
340,STOP,Visitor Timeout,NaN
339,PENALTY,John Tavares Hooking against Connor Brown,NaN
338,BLOCKED_SHOT,Jason Spezza shot blocked shot by Nick Holden,NaN


In [95]:
# game_id_tbox = widgets.Text(description="game_id", width=300)
# model_id_tbox = widgets.Text(description="model_id", width=300)
# button_update = widgets.Button(description="Update Dashboard")
# button_update.on_click(on_button_update_clicked)
# output = widgets.Output()

In [96]:
# print(err_msg)
# 



# return err_msg, components


In [97]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(game_tracker.events[['type', 'shooter_team_name','predictions']])

In [98]:
display(button_update)
# display(widgets.HBox((game_id_tbox, model_id_tbox, button_update)))
display(output)
# 2021020001, xgboost-feats-non-corr

NameError: name 'button_update' is not defined